#Challenge 4a: Simple knapsack problem with QAOA


Dynamic Programming Approach

In [1]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit import Aer
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
import numpy as np

In [2]:
val = [5,6,7,8,9]
wt = [4,5,6,7,8]
W = 18

In [3]:
def dp(W, wt, val, n):
    k = [[0 for x in range(W + 1)] for x in range(n + 1)]
    for i in range(n + 1):
        for w in range(W + 1):
            if i == 0 or w == 0:
                k[i][w] = 0
            elif wt[i-1] <= w:
                k[i][w] = max(val[i-1] + k[i-1][w-wt[i-1]], k[i-1][w])
            else:
                k[i][w] = k[i-1][w]
                
    picks=[0 for x in range(n)]
    volume=W
    for i in range(n,-1,-1):
        if (k[i][volume]>k[i-1][volume]):
            picks[i-1]=1
            volume -= wt[i-1]
    return k[n][W],picks

n = len(val)
print("optimal value:", dp(W, wt, val, n)[0])
print('\n index of the chosen items:')
for i in range(n): 
    if dp(W, wt, val, n)[1][i]: 
        print(i,end=' ')

optimal value: 21

 index of the chosen items:
1 2 3 

QAOA approach

In [4]:
# import packages necessary for application classes.
from qiskit_optimization.applications import Knapsack

In [5]:
def knapsack_quadratic_program():
    # Put values, weights and max_weight parameter for the Knapsack()
    
    ##############################
    # Provide your code here
    
    prob = Knapsack(values=[4,5,6,7,8], weights=[5,6,7,8,9], max_weight=18)
    #
    ##############################
    
    # to_quadratic_program generates a corresponding QuadraticProgram of the instance of the knapsack problem.
    kqp = prob.to_quadratic_program()
    return prob, kqp

prob,quadratic_program=knapsack_quadratic_program()
quadratic_program

<QuadraticProgram: maximize 4*x_0 + 5*x_1 + 6*x_2 + 7*x_3 + 8*x_4, 5 variables, 1 constraints, 'Knapsack'>

In [6]:
# Numpy Eigensolver
meo = MinimumEigenOptimizer(min_eigen_solver=NumPyMinimumEigensolver())
result = meo.solve(quadratic_program)
print('result:\n', result)
print('\n index of the chosen items:', prob.interpret(result)) 

result:
 fval=15.0, x_0=0.0, x_1=0.0, x_2=0.0, x_3=1.0, x_4=1.0, status=SUCCESS

 index of the chosen items: [3, 4]


In [7]:
# QAOA
seed = 123
algorithm_globals.random_seed = seed
qins = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), shots=1000, seed_simulator=seed, seed_transpiler=seed)

meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, quantum_instance=qins))
result = meo.solve(quadratic_program)
print('result:\n', result)
print('\n index of the chosen items:', prob.interpret(result)) 

result:
 fval=15.0, x_0=0.0, x_1=0.0, x_2=0.0, x_3=1.0, x_4=1.0, status=SUCCESS

 index of the chosen items: [3, 4]


Battery Revenue Optimization Problem

#Challenge 4b: Battery revenue optimization with Qiskit knapsack class

In [8]:
L1 = [5,3,3,6,9,7,1]
L2 = [8,4,5,12,10,11,2]
C1 = [1,1,2,1,1,1,2]
C2 = [3,2,3,2,4,3,3]
C_max = 16

In [9]:
values=[]
weights=[]
for i in range(len(L1)):
    values.append(L2[i]-L1[i])
for i in range(len(C1)):
    weights.append(C2[i]-C1[i])
max_weight=C_max-sum(C1)

In [10]:
def knapsack_argument(L1, L2, C1, C2, C_max):
      
    ##############################
    # Provide your code here
    values=[]
    weights=[]
    for i in range(len(L1)):
        values.append(L2[i]-L1[i])
    for i in range(len(C1)):
        weights.append(C2[i]-C1[i])
    max_weight=C_max-sum(C1)
    #
    ##############################
    return values, weights, max_weight
    
values, weights, max_weight = knapsack_argument(L1, L2, C1, C2, C_max)
print(values, weights, max_weight)
prob = Knapsack(values = values, weights = weights, max_weight = max_weight)
qp = prob.to_quadratic_program()
qp

[3, 1, 2, 6, 1, 4, 1] [2, 1, 1, 1, 3, 2, 1] 7


<QuadraticProgram: maximize 3*x_0 + x_1 + 2*x_2 + 6*x_3 + x_4 + 4*x_5 + x_6, 7 variables, 1 constraints, 'Knapsack'>

In [11]:
# QAOA
seed = 123
algorithm_globals.random_seed = seed
qins = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), shots=1000, seed_simulator=seed, seed_transpiler=seed)

meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, quantum_instance=qins))
result = meo.solve(qp)
print('result:', result.x)

item = np.array(result.x)
revenue=0
for i in range(len(item)):
    if item[i]==0:
        revenue+=L1[i]
    else:
        revenue+=L2[i]

print('total revenue:', revenue)

result: [1. 1. 1. 1. 0. 1. 0.]
total revenue: 50


#Challenge 4c: Battery revenue optimization with adiabatic quantum computation